#### Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from gensim.models import Word2Vec, FastText
from datasets import load_dataset
import numpy as np
# import whatever you need

c:\Users\Mausumi\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Choose model

In [ ]:
chosen_embeddings = FastText.load("embeddings/cc.bn.300.model")
#chosen_embeddings = FastText.load("embeddings/ai4b_subset_sg.model")
#chosen_embeddings = FastText.load("embeddings/ai4b_subset_fair.model")

In [ ]:
def build_simple_embedding(gensim_model):
    wv = gensim_model.wv if hasattr(gensim_model, 'wv') else gensim_model
    
    # 1. Grab all Gensim weights at once
    gensim_weights = torch.FloatTensor(wv.vectors)
    
    # 2. Create weights for our 4 special tokens
    pad_weight = torch.zeros(1, wv.vector_size)                     # <PAD> gets zeros
    special_weights = torch.randn(3, wv.vector_size) * 0.1          # <BOS>, <EOS>, <UNK> get random noise
    
    # 3. Stack them on top of the Gensim weights
    # Order will be: PAD, BOS, EOS, UNK, [Gensim Vocabulary...]
    all_weights = torch.cat([pad_weight, special_weights, gensim_weights], dim=0)
    
    # 4. Build the embedding layer
    embedding_layer = nn.Embedding.from_pretrained(all_weights, freeze=False, padding_idx=0)
    
    # 5. Shift all original Gensim indices by 4 to account for the new tokens
    word2idx = {'<PAD>': 0, '<BOS>': 1, '<EOS>': 2, '<UNK>': 3}
    word2idx.update({word: idx + 4 for word, idx in wv.key_to_index.items()})
    
    return embedding_layer, word2idx

# --- Execution ---
embedding_layer, word_to_index = build_simple_embedding(chosen_embeddings)